In [11]:
import numpy as np
import matplotlib.pyplot as plt
import time
import psutil
from tenpy.models import SpinChain
from tenpy.algorithms import dmrg
from tenpy.algorithms.dmrg import DMRGEngine
from tenpy.networks.mps import MPS
from tenpy.networks.site import SpinHalfSite

In [14]:
# Set up the range of spins to test
spin_range = range(20, 101, 10)  # From 20 to 100 spins in steps of 10
heisenberg_memory_usage = []
heisenberg_runtime = []

# Function to estimate memory usage (in MB) for the current process
def get_memory_usage():
    process = psutil.Process()
    memory_info = process.memory_info()
    return memory_info.rss / (1024 ** 2)  # Convert bytes to MB

# Run DMRG for each system size and track metrics
for n_spins in spin_range:
    # Define Heisenberg model parameters
    model_params = {
        "S": 0.5,            # Spin-1/2 system
        "L": n_spins,        # Length of the spin chain
        "Jx": 1.0,           # Coupling in the x-direction
        "Jy": 1.0,           # Coupling in the y-direction
        "Jz": 1.0,           # Coupling in the z-direction
        "bc_MPS": "finite",  # Boundary conditions for MPS
    }
    
    # Initialize Heisenberg model using TenPy
    model = SpinChain(model_params)

    # Create an initial MPS (wavefunction) for the DMRG algorithm
    spin_site = SpinHalfSite()
    psi = MPS.from_product_state([spin_site] * n_spins, ["up"] * n_spins, bc="finite")
    
    # Track initial memory usage before DMRG
    initial_memory = get_memory_usage()
    
    # Track runtime for DMRG simulation
    start_time = time.time()
    
    # Run DMRG algorithm
    dmrg_params = {
        "mixer": True,              # Mixer to avoid local minima
        "trunc_params": {"svd_min": 1e-10},  # Truncation tolerance
        "chi_max": 100,             # Maximum bond dimension (limits memory usage)
    }
    
    # Initialize and run the DMRGEngine
    eng = dmrg(psi, model, dmrg_params)
    eng.run()  # Run DMRG in place; modifies `psi`
    
    runtime = time.time() - start_time  # Calculate runtime
    energy = eng.e_ground_state  # Access the ground state energy
    
    # Track memory usage after DMRG
    final_memory = get_memory_usage()
    memory_usage = final_memory - initial_memory  # Memory used by DMRG process in MB
    
    # Append metrics
    heisenberg_memory_usage.append(memory_usage)
    heisenberg_runtime.append(runtime)
    
    print(f"Spins: {n_spins}, Energy: {E:.4f}, Memory Usage: {memory_usage:.2f} MB, Runtime: {runtime:.2f} seconds")

# Plotting the memory usage and runtime as functions of the number of spins
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Plot memory usage
axs[0].plot(spin_range, heisenberg_memory_usage, label="Heisenberg Ground State (DMRG)", marker='o')
axs[0].set_xlabel("Number of Spins")
axs[0].set_ylabel("Memory Usage (MB)")
axs[0].set_title("Memory Usage for Heisenberg Ground State Preparation with DMRG")
axs[0].legend()

# Plot runtime
axs[1].plot(spin_range, heisenberg_runtime, label="Heisenberg Ground State (DMRG)", marker='o')
axs[1].set_xlabel("Number of Spins")
axs[1].set_ylabel("Runtime (seconds)")
axs[1].set_title("Runtime for Heisenberg Ground State Preparation with DMRG")
axs[1].legend()

plt.tight_layout()
plt.show()

TypeError: 'module' object is not callable